### Imports

In [13]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

pd.set_option('display.max_rows', 500)

In [3]:
#Use kernel: conda_pytorch_p36

import numpy as np
import pandas as pd

from dfply import *
import plotly_express as px

from pandas.api.types import is_numeric_dtype
from pandas import Series, DataFrame
from dataclasses import dataclass, field, InitVar
from typing import Any, AnyStr, Callable, Collection, Dict, Hashable, Iterator, List, Mapping, NewType, Optional
from enum import Enum, IntEnum

### Read data

In [6]:
#Unzip
train_transaction = pd.read_csv('train_transaction.csv.zip', compression='zip', header=0, sep=',', quotechar='"', error_bad_lines=False)
train_identity = pd.read_csv('train_identity.csv.zip', compression='zip', header=0, sep=',', quotechar='"', error_bad_lines=False)

test_transaction = pd.read_csv('test_transaction.csv.zip', compression='zip', header=0, sep=',', quotechar='"', error_bad_lines=False)
test_identity = pd.read_csv('test_identity.csv.zip', compression='zip', header=0, sep=',', quotechar='"', error_bad_lines=False)

In [8]:
print("Shape of input dataframes")
print("train_transcation:", train_transaction.shape)
print("test_transcation:", test_transaction.shape)
print("train_identity:", train_identity.shape)
print("test_identity:", test_identity.shape)

Shape of input dataframes
train_transcation: (590540, 394)
test_transcation: (506691, 393)
train_identity: (144233, 41)
test_identity: (141907, 41)


In [14]:
train_transaction.head().T

,0,1,2,3,4
TransactionID,2987000,2987001,2987002,2987003,2987004
isFraud,0,0,0,0,0
TransactionDT,86400,86401,86469,86499,86506
TransactionAmt,68.5,29,59,50,50
ProductCD,W,W,W,W,H
card1,13926,2755,4663,18132,4497
card2,NaN,404,490,567,514
card3,150,150,150,150,150
card4,discover,mastercard,visa,mastercard,mastercard
card5,142,102,166,117,102


In [15]:
train_identity.head().T 

,0,1,2,3,4
TransactionID,2987004,2987008,2987010,2987011,2987016
id_01,0,-5,-5,-5,0
id_02,70787,98945,191631,221832,7460
id_03,NaN,NaN,0,NaN,0
id_04,NaN,NaN,0,NaN,0
id_05,NaN,0,0,0,1
id_06,NaN,-5,0,-6,0
id_07,NaN,NaN,NaN,NaN,NaN
id_08,NaN,NaN,NaN,NaN,NaN
id_09,NaN,NaN,0,NaN,0
